In [1]:
import torch 
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import random
import time
import os
import sys
import argparse
import copy
import math
import pickle
from collections import deque
from collections import namedtuple
from itertools import count
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime
from tqdm import tqdm
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch.utils.data import random_split
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from PIL import Image
from skimage import io
from skimage import transform
from skimage import color
from skimage import img_as_ubyte
from skimage import exposure
from skimage import util
from skimage import morphology
from skimage import filters
from skimage import feature

# Define the network
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, 1)
        self.conv2 = nn.Conv2d(16, 32, 3, 1)
        self.conv3 = nn.Conv2d(32, 64, 3, 1)
        self.fc1 = nn.Linear(64*6*6, 512)
        self.fc2 = nn.Linear(512, 2)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv3(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 64*6*6)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
# Define the dataset
class CustomDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img = self.data[idx][0]
        label = self.data[idx][1]
        if self.transform:
            img = self.transform(img)
        return img, label
    
# Define the training function
def train(model, device, train_loader, optimizer, criterion, epoch):
    model.train()
    running_loss = 0.0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    return running_loss / len(train_loader)

# Define the testing function
def test(model, device, test_loader, criterion):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader)
    accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, accuracy

# Define the main function
def main():
    # Define the device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Define the batch size
    batch_size = 64
    
    # Define the learning rate
    lr = 0.001
    
    # Define the number of epochs
    epochs = 10
    
    # Define the transform
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((64, 64)),
        transforms.ToTensor()
    ])
    
    # Load the data
    data = []
    for i in range(2):
        for j in range(100):
            img = io.imread(f"data/{i}/{j}.png")
            data.append((img, i))
    random.shuffle(data)
    train_data = data[:150]
    test_data = data[150:]
    train_dataset = CustomDataset(train_data, transform)
    test_dataset = CustomDataset(test_data, transform)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    # Define the model
    model = Net().to(device)
    
    # Define the optimizer
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    # Define the criterion
    criterion = nn.CrossEntropyLoss()
    
    # Train the model
    for epoch in range(epochs):
        train_loss = train(model, device, train_loader, optimizer, criterion, epoch)
        test_loss, accuracy = test(model, device, test_loader, criterion)
        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, Accuracy: {accuracy:.2f}%")
    
    # Save the model
    torch.save(model.state_dict(), "model.pth")
    
if __name__ == "__main__":
    main()

ModuleNotFoundError: No module named 'torch'